# STT와 TTS를 결합한 소음 제거 모델 (Noise Reduction Model Combining STT and TTS)

- 아래 코드는 논문에서 구현된 모델을 직접 테스트해볼 수 있도록 구현되었습니다.

# 1. Load STT Model

In [43]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer
from datasets import load_dataset

import librosa
import openai
openai.api_key = "(Insert your api key)" # OpenAI API key를 " " 사이에 넣어주세요.

import time
from gtts import gTTS
import IPython.display as ipd
import torch

In [44]:
model = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-noising_6") # Final Model

processor = WhisperProcessor.from_pretrained("openai/whisper-small")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

# 2. Build Model Pipeline

In [39]:
import os

def DNS(audio_path):
    directory, filename = os.path.split(audio_path)
    audio_dns_path = directory + "/dns/" + filename
    command = f"python ./noise-reduction-master/denoise.py --model=./noise-reduction-master/models/tscn --noisy={audio_path} --denoise={audio_dns_path}"
    # 명령어 실행
    exit_code = os.system(command)
    # 종료 코드 확인
    if exit_code != 0:
        print(f"Error: [Errno {exit_code}]")
        print("DNS: Fail!")
    else:
        print("DNS: Done!")
    return "audio/" + "dns/" + filename

def STT(audio_dns_path, model=model):
    y, s = librosa.load(audio_dns_path, sr=16000) # sampling rate 변경 (44100 -> 16000)
    input_features = processor(y, sampling_rate=16000, return_tensors="pt").input_features
    predicted_ids = model.generate(input_features.to('cuda'), forced_decoder_ids=forced_decoder_ids)
    predicted = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    print(f"STT result: {predicted}")
    return predicted

def GPT(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "주어진 문장에서 틀린 글자와 단어를 맥락에 맞게 모두 고치세요."},
            {"role": "system", "content": "구두점(마침표, 쉼표, 느낌표, 물음표)이 표기되어 있지 않으면 구두점을 추가로 표기하세요."},
            {"role": "system", "content": "단, 고친 문장만을 출력하세요. 다른 어떠한 부가적인 문장 추가하지 마세요."},
            {"role": "user", "content": text},
        ],
        temperature=0.1, # temperature 설정 (높은 값: 출력이 무작위로 생성, 낮은 값: 출력이 일관성 있게 생성)
    )

    output_text = response["choices"][0]["message"]["content"]
    print(f"ChatGPT result: {output_text}")
    return output_text

def TTS(text, audio_path):
    directory, filename = os.path.split(audio_path)
    save_path = "audio/" + "result/" + filename
    tts = gTTS(text=text, lang='ko')
    tts.save(save_path)
    print("TTS result: Done!")

In [40]:
def main(audio_path, model=model, GPT_flag=True):
    start = time.time()
    
    # DNS
    audio_dns_path = DNS(audio_path)

    # STT
    text = STT(audio_dns_path, model)

    # GPT
    if GPT_flag == True:
        text = GPT(text)

    # TTS
    TTS(text, audio_path)
    
    end = time.time()
    print(f"Inference time: {end-start:.1f}")

In [41]:
main("Insert your audio path") # 오디오 wav 파일을 audio 폴더에 넣고 " "에 전체 경로를 넣어주세요.

DNS: Done!
STT result: 최근 블루투스 이어폰의 사용의 증가함에 따라 이어폰을 착용하며 통화하는 사람들이 늘어나고 있다. 그러나 블루투스 이어폰의 특성상 주변 소음을 많이 포함하기도 해 통화 품질의 영향을 미치게 된다.
ChatGPT result: 최근 블루투스 이어폰의 사용이 증가함에 따라 이어폰을 착용하며 통화하는 사람들이 늘어나고 있다. 그러나 블루투스 이어폰의 특성상 주변 소음을 많이 포함하기도 해 통화 품질에 영향을 미치게 된다.
TTS result: Done!
Inference time: 11.9
